In [1]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
from dk_weekly_update import DkWeeklyUpdate as dk

In [2]:
df = pd.read_csv('input/wr/wr_loc_cleaned.csv')
df['year'].unique()

array([2012., 2014., 2015., 2016., 2017., 2018., 2013.])

In [4]:
df[df['year']==2018]['Week'].unique()

array([1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [5]:
def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')
def_2018.head()


,Rank,Name,Team,Position,Week,Opponent,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
0,1,Chicago Bears,CHI,DST,8,NYJ,12.32,10.0,20.2,8.6,5,56.12
1,2,Arizona Cardinals,ARI,DST,8,SF,12.50,9.8,28.3,10.7,7,68.30
2,3,New England Patriots,NE,DST,8,BUF,10.42,22.0,29.0,9.0,9,79.42
3,4,Pittsburgh Steelers,PIT,DST,8,CLE,15.10,16.5,29.6,12.4,7,80.60
4,5,Minnesota Vikings,MIN,DST,8,NO,10.86,38.8,17.9,0.0,16,83.56


In [6]:
df.sort_values(['year','Week'],inplace=True)
df.tail()

,year,games,Name,Team,Week,Opponent,tar,rec,yds,yds/rec,yds/tar,td,def_rk,h/a_1,surface_0,points
15825,2018.0,11,Victor Bolden Jr.,SF,6,GB,0,0,0,0.0,0.0,0,13,0.0,1.0,0.0
15826,2018.0,29,Will Fuller,HOU,6,BUF,3,2,33,16.5,11.0,0,6,1.0,0.0,5.3
15827,2018.0,47,Willie Snead,BAL,6,TEN,10,7,60,8.6,6.0,0,18,0.0,1.0,13.0
15828,2018.0,5,Zach Pascal,IND,6,NYJ,7,5,35,7.0,5.0,0,27,0.0,0.0,9.3
15829,2018.0,21,Zay Jones,BUF,6,HOU,8,3,35,11.7,4.4,1,7,0.0,0.0,12.5


In [6]:
wk8 = pd.read_csv('input/wr/wr_wk9.csv')

wk8.head()

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,ReceivingYardsPerTarget,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameDraftKings,FantasyPointsDraftKings
0,1,Marvin Jones,DET,WR,8,SEA,10,7,70.0,117,...,11.7,16.7,0,0,0.0,0,0,0,33.7,33.7
1,1,Sammy Watkins,KC,WR,8,DEN,9,8,88.9,107,...,11.9,13.4,0,0,0.0,0,0,0,33.7,33.7
2,3,Mike Evans,TB,WR,8,CIN,13,6,46.2,179,...,13.8,29.8,0,0,0.0,0,0,0,32.9,32.9
3,4,Tyler Boyd,CIN,WR,8,TB,10,9,90.0,138,...,13.8,15.3,1,-2,-2.0,0,0,0,31.6,31.6
4,5,Stefon Diggs,MIN,WR,8,NO,11,10,90.9,119,...,10.8,11.9,0,0,0.0,0,0,0,30.9,30.9


In [7]:
location = pd.read_csv('input/location.csv')
location.tail()

,Unnamed: 0,date,day,week,home,away,year
1653,116,"October 28, 2018",Sun,8,OAK,IND,2018
1654,117,"October 28, 2018",Sun,8,LAR,GB,2018
1655,118,"October 28, 2018",Sun,8,ARI,SF,2018
1656,119,"October 28, 2018",Sun,8,MIN,NO,2018
1657,120,"October 29, 2018",Mon,8,BUF,NE,2018


In [8]:
position_dict = {'qb':'DraftKingsQuarterbackFantasyPointsAllowedAverage',
                    'wr':'DraftKingsWideReceiverFantasyPointsAllowedAverage',
                    'rb':'DraftKingsRunningbackFantasyPointsAllowedAverage',
                    'te':'DraftKingsTightEndFantasyPointsAllowedAverage'}

team_dict = {'New England Patriots':'NE',
            'Green Bay Packers':'GB',
            'Tampa Bay Buccaneers':'TB',
            'Kansas City Chiefs':'KC',
            'San Diego Chargers':'SD',
            'St. Louis Rams':'STL',
            'New Orleans Saints':'NO',
            'New York Jets':'NYJ',
            'New York Giants':'NYG',
            'San Francisco 49ers':'SF',
            'Los Angeles Rams':'LAR',
            'Jacksonville Jaguars':'JAX',
            'Los Angeles Chargers':'LAC'}

In [18]:
df.head()

,year,games,Name,Team,Week,Opponent,tar,rec,yds,yds/rec,yds/tar,td,def_rk,h/a_1,surface_0,points
0,2012.0,1,AJ Green,CIN,1,BAL,11,5,70,14.0,6.4,0,30,0.0,1.0,7.0
1,2012.0,1,Aldrick Robinson,WAS,1,NO,6,4,52,13.0,8.7,1,25,0.0,0.0,11.2
2,2012.0,1,Alshon Jeffery,CHI,1,IND,5,3,80,26.7,16.0,1,2,1.0,1.0,14.0
3,2012.0,1,Andre Holmes,DAL,1,NYG,0,0,0,0.0,0.0,0,11,0.0,0.0,0.0
4,2012.0,1,Andre Johnson,HOU,1,MIA,10,8,119,14.9,11.9,1,22,1.0,0.0,17.9


In [9]:
wk8.rename(columns ={  'Receptions':'rec',
                     'ReceivingTargets':'tar',
                      'ReceivingYards':'yds',
                     'ReceivingYardsPerReception':'yds/rec',
                     'ReceivingYardsPerTarget':'yds/tar',
                      'ReceivingTouchdowns':'td',                    
                      'FantasyPointsDraftKings':'points'},inplace=True)
wk8.columns


Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent', 'tar', 'rec',
       'ReceptionPercentage', 'yds', 'td', 'ReceivingLong', 'yds/tar',
       'yds/rec', 'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsPerGameDraftKings', 'points'],
      dtype='object')

In [66]:
wr.head()

,games,year,games,Name,Team,Week,Opponent,tar,rec,yds,yds/rec,yds/tar,td,def_rk,h/a_1,surface_0,points
0,1,2012.0,2,AJ Green,CIN,1,BAL,11,5,70,14.0,6.4,0,30,0.0,1.0,7.0
1,2,2012.0,3,AJ Green,CIN,2,CLE,12,7,58,8.3,4.8,1,16,1.0,0.0,11.8
2,3,2012.0,4,AJ Green,CIN,3,WAS,11,9,183,20.3,16.6,1,28,0.0,1.0,25.4
3,4,2012.0,5,AJ Green,CIN,4,JAX,9,6,117,19.5,13.0,1,15,0.0,1.0,17.7
4,5,2012.0,6,AJ Green,CIN,5,MIA,13,9,65,7.2,5.0,1,22,1.0,0.0,12.5


In [67]:
df.columns

Index(['year', 'games', 'Name', 'Week', 'Opponent', 'Team', 'tar', 'rec',
       'yds', 'yds/rec', 'yds/tar', 'td', 'def_rk', 'h/a_1', 'surface_0',
       'points'],
      dtype='object')

In [10]:
ru_cols = [col for col in wk8.columns if 'Rushing'in col]
wk8.drop(ru_cols, axis=1, inplace=True)

In [11]:
wk8.drop(['FantasyPointsPerGameDraftKings', 'Rank','Position', 'Fumbles','FumblesLost','ReceptionPercentage','ReceptionPercentage','ReceivingLong'],axis=1, inplace=True)
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'tar', 'rec', 'yds', 'td',
       'yds/tar', 'yds/rec', 'points'],
      dtype='object')

In [70]:
def_2018.head()

,Rank,Name,Team,Position,Played,DraftKingsQuarterbackFantasyPointsAllowedAverage,DraftKingsRunningbackFantasyPointsAllowedAverage,DraftKingsWideReceiverFantasyPointsAllowedAverage,DraftKingsTightEndFantasyPointsAllowedAverage,DraftKingsKickerFantasyPointsAllowedAverage,DraftKingsFantasyPointsAllowedAverage
Team,,,,,,,,,,,
JAX,1,Jacksonville Jaguars,JAX,DST,7,15.37,20.50,29.46,8.50,7.14,80.97
DAL,2,Dallas Cowboys,DAL,DST,7,17.29,22.43,29.70,10.64,4.00,84.06
BAL,3,Baltimore Ravens,BAL,DST,7,14.60,14.57,32.21,11.77,2.57,75.73
BUF,4,Buffalo Bills,BUF,DST,7,16.44,28.20,32.59,10.86,6.14,94.23
MIN,5,Minnesota Vikings,MIN,DST,7,21.11,18.99,33.73,15.77,3.86,93.46


In [12]:
wk8.head()


,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points
0,Marvin Jones,DET,8,SEA,10,7,117,2,11.7,16.7,33.7
1,Sammy Watkins,KC,8,DEN,9,8,107,2,11.9,13.4,33.7
2,Mike Evans,TB,8,CIN,13,6,179,1,13.8,29.8,32.9
3,Tyler Boyd,CIN,8,TB,10,9,138,1,13.8,15.3,31.6
4,Stefon Diggs,MIN,8,NO,11,10,119,1,10.8,11.9,30.9


In [13]:
df.shape

(15830, 16)

## Add Defensive Ranking

In [17]:
wk8 = add_defensive_rank(wk8, def_2018, 'wr')
wk8

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,def_rk
0,Marvin Jones,DET,8,SEA,10,7,117,2,11.7,16.7,33.70,23
1,Sammy Watkins,KC,8,DEN,9,8,107,2,11.9,13.4,33.70,21
2,Mike Evans,TB,8,CIN,13,6,179,1,13.8,29.8,32.90,28
3,Tyler Boyd,CIN,8,TB,10,9,138,1,13.8,15.3,31.60,25
4,Stefon Diggs,MIN,8,NO,11,10,119,1,10.8,11.9,30.90,27
5,Larry Fitzgerald,ARI,8,SF,12,8,102,1,8.5,12.8,29.20,22
6,Adam Thielen,MIN,8,NO,7,7,103,1,14.7,14.7,26.80,27
7,Will Fuller,HOU,8,MIA,6,5,124,1,20.7,24.8,26.40,26
8,DeAndre Hopkins,HOU,8,MIA,7,6,82,2,11.7,13.7,26.20,26
9,Antonio Brown,PIT,8,CLE,8,6,74,2,9.2,12.3,25.40,14


In [18]:
loc = location[location['year']==2018]
loc = loc[loc['week']==8]
loc.shape

(14, 7)

## Home or Away

In [19]:
loc_wk = {}
home_lst = []
for week in loc['week'].unique():
    loc_wk[week]  = loc[loc['week']==week]
for idx, team in enumerate(wk8['Team']):
    if team in list(loc['home']):
        home_lst.append('home')           
    else:
        home_lst.append('away')
wk8['h/a'] = home_lst

In [20]:
wk8

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,def_rk,h/a
0,Marvin Jones,DET,8,SEA,10,7,117,2,11.7,16.7,33.70,23,home
1,Sammy Watkins,KC,8,DEN,9,8,107,2,11.9,13.4,33.70,21,home
2,Mike Evans,TB,8,CIN,13,6,179,1,13.8,29.8,32.90,28,away
3,Tyler Boyd,CIN,8,TB,10,9,138,1,13.8,15.3,31.60,25,home
4,Stefon Diggs,MIN,8,NO,11,10,119,1,10.8,11.9,30.90,27,home
5,Larry Fitzgerald,ARI,8,SF,12,8,102,1,8.5,12.8,29.20,22,home
6,Adam Thielen,MIN,8,NO,7,7,103,1,14.7,14.7,26.80,27,home
7,Will Fuller,HOU,8,MIA,6,5,124,1,20.7,24.8,26.40,26,home
8,DeAndre Hopkins,HOU,8,MIA,7,6,82,2,11.7,13.7,26.20,26,home
9,Antonio Brown,PIT,8,CLE,8,6,74,2,9.2,12.3,25.40,14,home


## Surface

In [21]:
surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [22]:

# Add the type of field the player palyed on by mapping the field on which the game was played to the surface dictionary

surface = []
for idx, team in enumerate(wk8['Team']):
    if wk8.iloc[idx]['h/a']=='home':
        surface.append(surface_dict[team])
    else:
        surface.append(surface_dict[wk8.iloc[idx]['Opponent']])


wk8['surface'] = surface

In [25]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'tar', 'rec', 'yds', 'td',
       'yds/tar', 'yds/rec', 'points', 'def_rk', 'h/a', 'surface'],
      dtype='object')

In [27]:
wk8.reset_index(inplace=True, drop=True)
wk8

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,def_rk,h/a,surface
0,Marvin Jones,DET,8,SEA,10,7,117,2,11.7,16.7,33.70,23,home,turf
1,Sammy Watkins,KC,8,DEN,9,8,107,2,11.9,13.4,33.70,21,home,grass
2,Mike Evans,TB,8,CIN,13,6,179,1,13.8,29.8,32.90,28,away,turf
3,Tyler Boyd,CIN,8,TB,10,9,138,1,13.8,15.3,31.60,25,home,turf
4,Stefon Diggs,MIN,8,NO,11,10,119,1,10.8,11.9,30.90,27,home,turf
5,Larry Fitzgerald,ARI,8,SF,12,8,102,1,8.5,12.8,29.20,22,home,grass
6,Adam Thielen,MIN,8,NO,7,7,103,1,14.7,14.7,26.80,27,home,turf
7,Will Fuller,HOU,8,MIA,6,5,124,1,20.7,24.8,26.40,26,home,turf
8,DeAndre Hopkins,HOU,8,MIA,7,6,82,2,11.7,13.7,26.20,26,home,turf
9,Antonio Brown,PIT,8,CLE,8,6,74,2,9.2,12.3,25.40,14,home,grass


## Encode

In [28]:
wk8 = one_hot(wk8)

In [29]:
wk8.drop(['h/a_0','surface_1','surface','h/a','surface_encoded','h/a_encoded'], axis=1, inplace=True)
wk8.shape

(140, 14)

In [30]:
wk8.columns

Index(['Name', 'Team', 'Week', 'Opponent', 'tar', 'rec', 'yds', 'td',
       'yds/tar', 'yds/rec', 'points', 'def_rk', 'h/a_1', 'surface_0'],
      dtype='object')

In [31]:
wk8['year']=2018
wk8.shape

(140, 15)

In [32]:
df.shape

(15830, 16)

## Games

In [33]:
games_lst = []
temp = wk8.set_index('Name')
for name in temp.index:
    if name in list(df['Name'].unique()):
        games_lst.append(df[df['Name']==name]['games'].max()+1)
    else:
        games_lst.append(1)
    

In [34]:
wk8['games'] = games_lst

In [35]:
wk8.head()

,Name,Team,Week,Opponent,tar,rec,yds,td,yds/tar,yds/rec,points,def_rk,h/a_1,surface_0,year,games
0,Marvin Jones,DET,8,SEA,10,7,117,2,11.7,16.7,33.7,23,1.0,0.0,2018,81
1,Sammy Watkins,KC,8,DEN,9,8,107,2,11.9,13.4,33.7,21,1.0,1.0,2018,59
2,Mike Evans,TB,8,CIN,13,6,179,1,13.8,29.8,32.9,28,0.0,0.0,2018,68
3,Tyler Boyd,CIN,8,TB,10,9,138,1,13.8,15.3,31.6,25,1.0,0.0,2018,34
4,Stefon Diggs,MIN,8,NO,11,10,119,1,10.8,11.9,30.9,27,1.0,0.0,2018,48


## Reshape

In [36]:
df.columns

Index(['year', 'games', 'Name', 'Week', 'Opponent', 'Team', 'tar', 'rec',
       'yds', 'yds/rec', 'yds/tar', 'td', 'def_rk', 'h/a_1', 'surface_0',
       'points'],
      dtype='object')

In [41]:
cols = wk8.columns.tolist()
cols = cols[-2:] + cols[:1] + cols[2:4] + cols[1:2] + cols[4:7] + cols[9:10] +cols[8:9]+ cols[7:8] +cols[-5:-2] + cols[-6:-5]
wk8 = wk8[cols]
wk8.head()

,year,games,Name,Week,Opponent,Team,tar,rec,yds,yds/rec,yds/tar,td,def_rk,h/a_1,surface_0,points
0,2018,81,Marvin Jones,8,SEA,DET,10,7,117,16.7,11.7,2,23,1.0,0.0,33.7
1,2018,59,Sammy Watkins,8,DEN,KC,9,8,107,13.4,11.9,2,21,1.0,1.0,33.7
2,2018,68,Mike Evans,8,CIN,TB,13,6,179,29.8,13.8,1,28,0.0,0.0,32.9
3,2018,34,Tyler Boyd,8,TB,CIN,10,9,138,15.3,13.8,1,25,1.0,0.0,31.6
4,2018,48,Stefon Diggs,8,NO,MIN,11,10,119,11.9,10.8,1,27,1.0,0.0,30.9


In [42]:
df.shape

(15830, 16)

In [43]:
wk8.columns

Index(['year', 'games', 'Name', 'Week', 'Opponent', 'Team', 'tar', 'rec',
       'yds', 'yds/rec', 'yds/tar', 'td', 'def_rk', 'h/a_1', 'surface_0',
       'points'],
      dtype='object')

## Concatenate

In [44]:
wr = pd.concat([df,wk8])
wr.shape

(15970, 16)

In [45]:
wr.tail()

,year,games,Name,Week,Opponent,Team,tar,rec,yds,yds/rec,yds/tar,td,def_rk,h/a_1,surface_0,points
135,2018.0,21,Justin Hardee,8,MIN,NO,0,0,0,0.0,0.0,0,1,0.0,0.0,0.0
136,2018.0,3,Dan Arnold,8,MIN,NO,0,0,0,0.0,0.0,0,1,0.0,0.0,0.0
137,2018.0,8,Brandon Zylstra,8,NO,MIN,0,0,0,0.0,0.0,0,27,1.0,0.0,0.0
138,2018.0,88,Matthew Slater,8,BUF,NE,0,0,0,0.0,0.0,0,19,0.0,0.0,0.0
139,2018.0,49,Phillip Dorsett,8,BUF,NE,0,0,0,0.0,0.0,0,19,0.0,0.0,0.0


In [46]:
wr.to_csv('input/wr/wr_loc_cleaned.csv', index=False)

In [93]:
# Parse upcoming week file by position

def clean_current(csv):# Read in files

    df_upcoming = pd.read_csv(csv)
    def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

    # Obtain home and away attributes
    df_upcoming['year']=2018
    df_upcoming['home'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[1]).split()[0])
    df_upcoming['away'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[0]))

    rb = []
    wr = []
    te = []
    qb= []
    for idx,pos in enumerate(df_upcoming['Position']):
        if pos == 'RB':
            rb.append(idx)
        elif pos =='WR':
            wr.append(idx)
        elif pos =='TE':
            te.append(idx)
        elif pos =='QB':
            qb.append(idx)


    # Sort current week dataframe

    qb_df = df_upcoming.iloc[qb]
    rb_df = df_upcoming.iloc[rb]
    wr_df = df_upcoming.iloc[wr]
    te_df = df_upcoming.iloc[te]
    
    return qb_df,wr_df,te_df, rb_df

In [14]:
# Prepare defense dataframe by sorting by relevant attribute reseting the indexand using rank as the reset dataframe index +1

def add_defensive_rank(df, df1, position):
    df1.sort_values(position_dict[position], inplace=True)
    df1.reset_index(inplace=True, drop=True)
    df1['Rank']= df1.index+1
    df1.index=df1['Team']

# Map defensive rnking to eack player in the statist  ical dataframe 

    df['def_rk'] = list(map(lambda x: df1.loc[x]['Rank'], df['Opponent']))
    
    return df


In [15]:
# A function designed to add the Draftkings fantasy football salary to every players statistics dataframe

def add_salary(df,df2):
    df = df.sort_values(['Week','PlayerID'])
    df = df.drop(['FantasyPointsPerGameDraftKings','Rank','Position'], axis=1)
    df.reset_index(inplace=True, drop=True)
    df2 = df2.sort_values(['Week','PlayerID'])
    df2 = df2.drop(['Rank','Position'], axis=1)
    df2.reset_index(inplace=True, drop=True)
    df['salary'] = df2['DraftKingsSalary']
    
    return df

    

In [16]:
# One hot encode all non numeric attributes


def one_hot(df):
    df.reset_index(inplace=True, drop=True)
    encoder = LabelEncoder()
    df['h/a_encoded'] = encoder.fit_transform(df['h/a'])
    df['surface_encoded'] = encoder.fit_transform(df['surface'])


    home =OneHotEncoder()
    surface = OneHotEncoder()

    X = home.fit_transform(df['h/a_encoded'].values.reshape(-1,1)).toarray()
    Xs = surface.fit_transform(df['surface_encoded'].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["h/a_"+str(int(i)) for i in range(X.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)


    dfOneHot = pd.DataFrame(Xs, columns = ['surface_'+str(int(i)) for i in range(Xs.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)

    return df

In [97]:
csv = 'input/qb/DKSalaries.csv'
qb7, wr7, te7, rb7 = clean_current(csv)
